## Importation de bibliotheque

In [4]:
from datetime import datetime
import pandas as pd
import os

import gspread
from gspread_dataframe import set_with_dataframe
from modules.sippec.sippecapi import SippecAPI
from google.oauth2.service_account import Credentials

In [ ]:
CREDENTIALS_PATH = '/Users/jeanmermozeffi/DataspellProjects/scraping-master/credential/secret-reporting-sheet.json'
credentials = Credentials.from_service_account_file(CREDENTIALS_PATH, scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])
gc = gspread.authorize(credentials)

In [2]:
sippec_api = SippecAPI()

In [ ]:
first_day_previous_month , last_day_previous_month = sippec_api.get_month_ranges()
first_day_previous_month , last_day_previous_month

In [ ]:
sippec_data = sippec_api.get_sippec_data()

In [ ]:
len(sippec_data)

In [ ]:
cleaned_data_category = sippec_api.process_datas_operators(sippec_data, 'datasCategorie')
cleaned_data_category

In [ ]:
df_cleaned_data_category = pd.DataFrame(cleaned_data_category)
df_cleaned_data_category.rename(columns={'libelle': 'Category'}, inplace=True)
df_cleaned_data_category.head()

In [ ]:
# df_cleaned_data_category['SommeNomreScan'] = df_cleaned_data_category['nbreScan'].sum()
# df_cleaned_data_category['SommeNomreGroupeProduction'] = df_cleaned_data_category['nbreGroupeProduction'].sum()

In [ ]:
# Vérifiez d'abord le type de données actuel de la colonne
if df_cleaned_data_category['Category'].dtype == 'object':
    df_cleaned_data_category['Category'] = df_cleaned_data_category['Category'].astype('string')

In [ ]:
df_cleaned_data_category = df_cleaned_data_category.drop(columns=['date'])

In [ ]:
df_cleaned_data_category

## Definition fonction

In [ ]:
df_exploded_category = sippec_api.explode_and_rename(df=df_cleaned_data_category, columns_col='Category', value_cols=['nbreScan', 'nbreGroupeProduction'])
df_exploded_category.head()

In [ ]:

categoty_file_path = 'Report SIPPEC Data Category.csv'
category_save_path = sippec_api.create_folder_if_not_exists(categoty_file_path)
df_cleaned_data_category.to_csv(category_save_path, index=False)

## Get Previous Data

In [ ]:
spreadsheet_name = "Reporting SIPPEC Academie Data"
sheet_name = "Report Sippec"
previous_data = sippec_api.get_previous_month_data(spreadsheet_name, sheet_name)

In [ ]:
previous_data

## Calculate Rate Data

In [ ]:
df_exploded_previous_data_category = sippec_api.explode_and_rename(df=previous_data, columns_col='libelle_category', value_cols=['nbreScan', 'nbreGroupeProduction'])

In [ ]:
SPREADSHEET_ACADEMIA_NAME = 'Reporting SIPPEC Academie Data'
SHEET_NAME = f"Reporting Sippec Data Month"
sippec_api.create_or_clear_worksheet(SPREADSHEET_ACADEMIA_NAME, SHEET_NAME, df_cleaned_data_category.iloc[:, :-1])

## data operators

In [ ]:
cleaned_data_operators = sippec_api.process_datas_operators(sippec_data, 'datasOperateur')
cleaned_data_operators

In [ ]:
df_cleaned_data_operator = pd.DataFrame(cleaned_data_operators)
df_cleaned_data_operator.rename(columns={'libelle': 'Operators'}, inplace=True)
df_cleaned_data_operator.head()

In [ ]:
current_montant_total = df_cleaned_data_operator['montantTransaction'].sum()
previous_montant_total = 595090

In [ ]:
rate_total = f"{round(((current_montant_total - previous_montant_total) / previous_montant_total) * 100, 2)}%"
rate_total 

In [ ]:
df_rate_montant = pd.DataFrame([rate_total], columns=['Taux Croissance Montant'])

In [ ]:
sippec_api.create_or_clear_worksheet(SPREADSHEET_ACADEMIA_NAME, SHEET_NAME, df_rate_montant, df_row=1, col_row=13)


In [ ]:
df_cleaned_data_operator['Taux Croissance'] = df_cleaned_data_operator['Taux Croissance']

In [ ]:
if df_cleaned_data_operator['Operators'].dtype == 'object':
    df_cleaned_data_operator['Operators'] = df_cleaned_data_operator['Operators'].astype('string')

In [ ]:
df_cleaned_data_operator.dtypes

In [ ]:
df_exploded_operator = sippec_api.explode_and_rename(df=df_cleaned_data_operator, columns_col='Operators', value_cols=['nbreTransaction', 'montantTransaction'])

In [ ]:
df_exploded_operator.head()

In [ ]:
df_cleaned_data_operator.iloc[:, :-1]

In [ ]:
operator_file_path = 'Report SIPPEC Data Operator.csv'
save_path_operator = sippec_api.create_folder_if_not_exists(operator_file_path)
df_cleaned_data_operator.to_csv(save_path_operator, index=False)

In [ ]:
SPREADSHEET_ACADEMIA_NAME = 'Reporting SIPPEC Academie Data'
SHEET_NAME = f"Reporting Sippec Data Month"

In [ ]:
sippec_api.create_or_clear_worksheet(SPREADSHEET_ACADEMIA_NAME, SHEET_NAME, df_cleaned_data_operator.iloc[:, :-1], df_row=1, col_row=5)

In [ ]:
ACTIVITY_COLUMN = ["ROYAL", "CHAMPION", "STAR", "AUTRE(S)", "PRINCE"]
OPERATOR_COLUMN = ["ORANGE", "MTN", "MOOV", "AUTRES"]
PRODUCTION_COLUMN = ["NOMBRE_SCAN", "NOMBRE_GROUPE_PRODUCTION"]
TRANSACTION_COLUMN = ["NOMBRE_TRANSACTION", "MONTANT_TRANSACTION"]
CATEGORY_KEY = "datasCategorie"
OPERATOR_KEY = "datasOperateur"
RAPPORT_COLUMN = ["ROYAL", "CHAMPION", "STAR", "AUTRE(S)", "PRINCE", "ORANGE", "MTN", "MOOV AFRICA", "AUTRE"]

## Ademician Data

In [ ]:
academician_data = sippec_api.get_academician_data()
academician_data

In [ ]:
df_academician_data = pd.DataFrame(academician_data)
df_academician_data = df_academician_data[['libelle', 'valeur', 'date', 'codeCouleur']]
df_academician_data.rename(columns={'libelle': 'Mode'}, inplace=True)
df_academician_data.rename(columns={'valeur': 'Academiciens'}, inplace=True)
df_academician_data.head()

In [ ]:
if df_academician_data['Mode'].dtype == 'object':
    df_academician_data['Mode'] = df_academician_data['Mode'].astype('string')

In [ ]:
df_exploded_academiciens = sippec_api.explode_and_rename(df=df_academician_data, columns_col='Mode', value_cols=['Academiciens'])

In [ ]:
df_exploded_academiciens.head()

In [ ]:
previous_data = df_exploded_academiciens.copy()
current_data = df_exploded_academiciens.copy()

In [ ]:
previous_data.loc[0, 'ACTIF_Academiciens'] = 100
previous_data.loc[0, 'BLOQUER_Academiciens'] = 160
previous_data.loc[0, 'EN VEILLE_Academiciens'] = 25
previous_data.loc[0, 'INACTIF_Academiciens'] = 2500
previous_data.loc[0, 'SEMI ACTIF_Academiciens'] = 78

In [ ]:
previous_data.head()

In [ ]:
ACADEMICIAN_FILE_PATH = 'Report SIPPEC Data Mode Activity.csv'
SAVE_ACADEMICIAN_FILE_PATH = sippec_api.create_folder_if_not_exists(ACADEMICIAN_FILE_PATH)
df_academician_data.to_csv(SAVE_ACADEMICIAN_FILE_PATH, index=False)

In [ ]:
sippec_api.create_or_clear_worksheet(SPREADSHEET_ACADEMIA_NAME, SHEET_NAME, df_academician_data.iloc[:, :-2], df_row=1, col_row=9)

## Rate 

In [ ]:
rates_columns = ['ACTIF_Academiciens', 'BLOQUER_Academiciens',
                 'EN VEILLE_Academiciens', 'INACTIF_Academiciens',
                 'SEMI ACTIF_Academiciens']

In [ ]:
rates = sippec_api.get_multiple_scan_growth_rates(current_data, previous_data, rates_columns)

In [ ]:
rates

In [ ]:
rates_cleaned = {}
for key, value in rates.items():
    _key = key.replace("_Academiciens", "")
    rates_cleaned[_key] = value

In [ ]:
rates_cleaned

In [ ]:
df_academician_data_rate = df_academician_data.iloc[:, :-2]
df_academician_data_rate

In [ ]:
df_academician_data_rate['Taux Croissance'] = df_academician_data_rate['Mode'].map(rates_cleaned)

In [ ]:
df_academician_data_rate

In [ ]:
sippec_api.create_or_clear_worksheet(SPREADSHEET_ACADEMIA_NAME, SHEET_NAME, df_academician_data_rate, df_row=1, col_row=9)

## Classement Academicien

### Montant

In [3]:
option = 'MONTANT_SCAN'
data_montant_rankings, date_montant_ranking = sippec_api.get_academic_ranking(option)

In [4]:
data_montant_rankings

[{'id': 3809,
  'matricule': 'SIP03809-21',
  'nom': 'DAKOURY',
  'ROYAL_SOMME_SCAN': 0,
  'ROYAL_TOTAL_SCAN': 0,
  'date_ranking': '01/04/2024',
  'CHAMPION_SOMME_SCAN': 92000,
  'CHAMPION_TOTAL_SCAN': 184,
  'STAR_SOMME_SCAN': 1000,
  'STAR_TOTAL_SCAN': 4,
  'AUTRE(S)_SOMME_SCAN': 0,
  'AUTRE(S)_TOTAL_SCAN': 0,
  'PRINCE_SOMME_SCAN': 0,
  'PRINCE_TOTAL_SCAN': 0},
 {'id': 3896,
  'matricule': 'SIP00062-22',
  'nom': 'HONSOU',
  'ROYAL_SOMME_SCAN': 80000,
  'ROYAL_TOTAL_SCAN': 80,
  'date_ranking': '01/04/2024',
  'CHAMPION_SOMME_SCAN': 0,
  'CHAMPION_TOTAL_SCAN': 0,
  'STAR_SOMME_SCAN': 0,
  'STAR_TOTAL_SCAN': 0,
  'AUTRE(S)_SOMME_SCAN': 0,
  'AUTRE(S)_TOTAL_SCAN': 0,
  'PRINCE_SOMME_SCAN': 0,
  'PRINCE_TOTAL_SCAN': 0},
 {'id': 2910,
  'matricule': 'SIP02910-20',
  'nom': 'DIARRA',
  'ROYAL_SOMME_SCAN': 24000,
  'ROYAL_TOTAL_SCAN': 24,
  'date_ranking': '01/04/2024',
  'CHAMPION_SOMME_SCAN': 0,
  'CHAMPION_TOTAL_SCAN': 0,
  'STAR_SOMME_SCAN': 0,
  'STAR_TOTAL_SCAN': 0,
  'AUTRE(S)_SOM

In [17]:
df_data_montant_rankings = pd.DataFrame(data_montant_rankings)

In [18]:
df_data_montant_rankings = df_data_montant_rankings.drop(['date_ranking', 'id'], axis=1)

In [19]:

df_data_montant_rankings.head()

,matricule,nom,ROYAL_SOMME_SCAN,ROYAL_TOTAL_SCAN,CHAMPION_SOMME_SCAN,CHAMPION_TOTAL_SCAN,STAR_SOMME_SCAN,STAR_TOTAL_SCAN,AUTRE(S)_SOMME_SCAN,AUTRE(S)_TOTAL_SCAN,PRINCE_SOMME_SCAN,PRINCE_TOTAL_SCAN
0,SIP03809-21,DAKOURY,0,0,92000,184,1000,4,0,0,0,0
1,SIP00062-22,HONSOU,80000,80,0,0,0,0,0,0,0,0
2,SIP02910-20,DIARRA,24000,24,0,0,0,0,0,0,15000,15
3,SIP00516-23,GRAH,11000,11,0,0,0,0,0,0,24000,24
4,SIP00138-23,KOFFI,15000,15,4000,8,11500,46,0,0,0,0


### Nombre

In [20]:
option = 'NBRE_SCAN'
data_nombre_rankings, date_nombre_ranking = sippec_api.get_academic_ranking(option)

In [21]:
data_nombre_rankings

[{'id': 3809,
  'matricule': 'SIP03809-21',
  'nom': 'DAKOURY',
  'ROYAL_SOMME_SCAN': 0,
  'ROYAL_TOTAL_SCAN': 0,
  'date_ranking': '01/04/2024',
  'CHAMPION_SOMME_SCAN': 92000,
  'CHAMPION_TOTAL_SCAN': 184,
  'STAR_SOMME_SCAN': 1000,
  'STAR_TOTAL_SCAN': 4,
  'AUTRE(S)_SOMME_SCAN': 0,
  'AUTRE(S)_TOTAL_SCAN': 0,
  'PRINCE_SOMME_SCAN': 0,
  'PRINCE_TOTAL_SCAN': 0},
 {'id': 3896,
  'matricule': 'SIP00062-22',
  'nom': 'HONSOU',
  'ROYAL_SOMME_SCAN': 80000,
  'ROYAL_TOTAL_SCAN': 80,
  'date_ranking': '01/04/2024',
  'CHAMPION_SOMME_SCAN': 0,
  'CHAMPION_TOTAL_SCAN': 0,
  'STAR_SOMME_SCAN': 0,
  'STAR_TOTAL_SCAN': 0,
  'AUTRE(S)_SOMME_SCAN': 0,
  'AUTRE(S)_TOTAL_SCAN': 0,
  'PRINCE_SOMME_SCAN': 0,
  'PRINCE_TOTAL_SCAN': 0},
 {'id': 4124,
  'matricule': 'SIP00138-23',
  'nom': 'KOFFI',
  'ROYAL_SOMME_SCAN': 15000,
  'ROYAL_TOTAL_SCAN': 15,
  'date_ranking': '01/04/2024',
  'CHAMPION_SOMME_SCAN': 4000,
  'CHAMPION_TOTAL_SCAN': 8,
  'STAR_SOMME_SCAN': 11500,
  'STAR_TOTAL_SCAN': 46,
  'AUTRE

In [23]:
df_data_nombre_rankings = pd.DataFrame(data_nombre_rankings)

In [24]:
df_data_nombre_rankings = df_data_nombre_rankings.drop(['date_ranking', 'id'], axis=1)

In [25]:
df_data_nombre_rankings.head()

,matricule,nom,ROYAL_SOMME_SCAN,ROYAL_TOTAL_SCAN,CHAMPION_SOMME_SCAN,CHAMPION_TOTAL_SCAN,STAR_SOMME_SCAN,STAR_TOTAL_SCAN,AUTRE(S)_SOMME_SCAN,AUTRE(S)_TOTAL_SCAN,PRINCE_SOMME_SCAN,PRINCE_TOTAL_SCAN
0,SIP03809-21,DAKOURY,0,0,92000,184,1000,4,0,0,0,0
1,SIP00062-22,HONSOU,80000,80,0,0,0,0,0,0,0,0
2,SIP00138-23,KOFFI,15000,15,4000,8,11500,46,0,0,0,0
3,SIP00061-24,KANTE,1000,1,24500,49,3000,12,0,0,0,0
4,SIP00288-23,ADJESSAN,0,0,0,0,12500,50,0,0,0,0


In [ ]:
# CLASSEMENT_ACADEMICIAN_FILE_PATH = 'Report SIPPEC Data Classement Academicien.csv'
# SAVE_CLASSEMENT_ACADEMICIAN_FILE_PATH = sippec_api.create_folder_if_not_exists(CLASSEMENT_ACADEMICIAN_FILE_PATH)
# df_data_nombre_rankings.to_csv(SAVE_CLASSEMENT_ACADEMICIAN_FILE_PATH, index=False)

In [ ]:
# df_data_rankings = pd.read_csv('Resultats/Report SIPPEC Data Classement Academicien.csv')

In [9]:
SPREADSHEET_NAME_CLASSEMENT = 'SIPPEC Classement Academicien Data'

In [29]:
SHEET_NAME_MONTANT = f"Classement_SCAN_MONTANT"
sippec_api.create_or_clear_worksheet(SPREADSHEET_NAME_CLASSEMENT, SHEET_NAME_MONTANT, df_data_montant_rankings)

In [30]:
SHEET_NAME_NOMBRE = f"Classement_SCAN_NOMBRE"
sippec_api.create_or_clear_worksheet(SPREADSHEET_NAME_CLASSEMENT, SHEET_NAME_NOMBRE, df_data_nombre_rankings)

## Importation fichiers csv data

In [3]:
df_cleaned_data_category = pd.read_csv('Resultats/Report SIPPEC Data Category.csv')
df_cleaned_data_category.head()

,Category,nbreScan,nbreGroupeProduction,date
0,ROYAL,390,1028,01/04/2024
1,CHAMPION,690,0,01/04/2024
2,STAR,551,0,01/04/2024
3,AUTRE(S),14,0,01/04/2024
4,PRINCE,72,514,01/04/2024


In [4]:
df_cleaned_data_operator = pd.read_csv('Resultats/Report SIPPEC Data Operator.csv')
df_cleaned_data_operator.head()

,Operators,nbreTransaction,montantTransaction,date
0,Orange,45,111500,01/04/2024
1,MTN,144,368250,01/04/2024
2,MOOV,62,116250,01/04/2024
3,Autre,16,49000,01/04/2024


In [5]:
df_academician_data = pd.read_csv('Resultats/Report SIPPEC Data Mode Activity.csv')
df_academician_data.head()

,Mode,Academiciens,date,codeCouleur
0,ACTIF,160,01/04/2024,#58D68D
1,BLOQUER,40,01/04/2024,#ff0000
2,EN VEILLE,62,01/04/2024,#fafa7d
3,INACTIF,1400,01/04/2024,#F1948A
4,SEMI ACTIF,78,01/04/2024,#36a2eb


## All Data 

In [6]:
SPREADSHEET_NAME = 'Reporting SIPPEC Academie Data'
SHEET_NAME_ALL = f"SIPPEC Academie Reporting Data ALL"

In [7]:
df_exploded_category = sippec_api.explode_and_rename(df=df_cleaned_data_category, columns_col='Category', value_cols=['nbreScan', 'nbreGroupeProduction'])

df_exploded_operator = sippec_api.explode_and_rename(df=df_cleaned_data_operator, columns_col='Operators', value_cols=['nbreTransaction', 'montantTransaction'])

df_exploded_academiciens = sippec_api.explode_and_rename(df=df_academician_data, columns_col='Mode', value_cols=['Academiciens'])

In [8]:
dfs = [df_exploded_category, df_exploded_operator.iloc[:, 1:], df_exploded_academiciens.iloc[:, 1:]]
df_combined = pd.concat(dfs, axis=1)

In [9]:
df_combined

,date,AUTRE(S)_nbreGroupeProduction,CHAMPION_nbreGroupeProduction,PRINCE_nbreGroupeProduction,ROYAL_nbreGroupeProduction,STAR_nbreGroupeProduction,AUTRE(S)_nbreScan,CHAMPION_nbreScan,PRINCE_nbreScan,ROYAL_nbreScan,...,Orange_montantTransaction,Autre_nbreTransaction,MOOV_nbreTransaction,MTN_nbreTransaction,Orange_nbreTransaction,ACTIF_Academiciens,BLOQUER_Academiciens,EN VEILLE_Academiciens,INACTIF_Academiciens,SEMI ACTIF_Academiciens
0,01/04/2024,0.0,0.0,514.0,1028.0,0.0,14.0,690.0,72.0,390.0,...,111500.0,16.0,62.0,144.0,45.0,160.0,40.0,62.0,1400.0,78.0


In [10]:
sippec_api.create_or_clear_worksheet(SPREADSHEET_NAME, SHEET_NAME_ALL, df_combined)

In [ ]:
df_combined.loc[0, 'date'] = '01/05/2024'

In [ ]:
df_combined

In [ ]:
sippec_api.create_or_clear_worksheet(gc, SPREADSHEET_NAME, SHEET_NAME, df_combined)

In [ ]:
SHEET_NAME_ALL = f"SIPPEC Academie Reporting Data ALL"

In [ ]:
df_month_04 = sippec_api.get_data_for_month(gc, SPREADSHEET_NAME, SHEET_NAME_ALL, 2024, 4)
df_month_05 = sippec_api.get_data_for_month(gc, SPREADSHEET_NAME, SHEET_NAME_ALL, 2024, 5)

In [ ]:
df_month_05

In [ ]:
df_columns = ['AUTRE(S)_nbreGroupeProduction',
              'CHAMPION_nbreGroupeProduction', 'PRINCE_nbreGroupeProduction',
              'ROYAL_nbreGroupeProduction', 'STAR_nbreGroupeProduction',
              'AUTRE(S)_nbreScan', 'CHAMPION_nbreScan', 'PRINCE_nbreScan',
              'ROYAL_nbreScan', 'STAR_nbreScan', 'Autre_montantTransaction',
              'MOOV_montantTransaction', 'MTN_montantTransaction',
              'Orange_montantTransaction', 'Autre_nbreTransaction',
              'MOOV_nbreTransaction', 'MTN_nbreTransaction', 'Orange_nbreTransaction',
              'ACTIF_academiciens', 'BLOQUER_academiciens', 'EN VEILLE_academiciens',
              'INACTIF_academiciens', 'SEMI ACTIF_academiciens']

In [ ]:
data_growth_rates = sippec_api.get_multiple_scan_growth_rates(df_month_04, df_month_05, df_columns)

In [ ]:
data_growth_rates

In [ ]:
df_data_growth_rates = pd.DataFrame(data_growth_rates,  index=[0])
df_data_growth_rates.head()

In [ ]:
sippec_api.create_or_clear_worksheet(gc=gc, spreadsheet_name=SPREADSHEET_NAME, sheet_name=SHEET_NAME, dataframe=df_data_growth_rates, df_row=10, include_header=True)